### Model Fitting using Training set sampled images

In [1]:
%matplotlib widget
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import glob
import cv2

import os
import sys  
import re
import shutil
sys.path.insert(0, '../dip_utils')

from matrix_utils import arr_info
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair, 
                       vis_triple)
%run watershed.ipynb
%run calibrate.ipynb
%run greyscale.ipynb
%run image_stacking.ipynb
%run whitep_elimination.ipynb
%run skew_correction.ipynb

In [2]:
path = '../image_set'
dest = '../stacked_new'
test = '../test_images'

In [3]:
files = glob.glob('%s/*' % dest)
for f in files:
    os.remove(f)

In [4]:
image_stacking(path, dest)

In [5]:
clear = '../stacked_new/NTU0.jpg'

In [29]:
def process_model(dest, img_lib):
    # Some threshold values
    # 500 50 80 350
    vert_thres = 500
    horz_thres = 50
    length = 80
    dist = 330   # distance between the two cuvettes
    
    markers_lib = dict()
    for img in img_lib:
        markers = watershed(dest, img)
        markers_lib[img] = markers

    bottom_lib = dict()
    left_lib = dict()
    top_lib = dict()
    right_lib = dict()

    # Acquire the bottom and the leftmost point of the ROI
    for key in markers_lib:
        bottom = np.where(markers_lib[key]==1)[0].max()
        bottom_lib[key] = bottom
        left = np.where((markers_lib[key]!=1) & (markers_lib[key]!=-1))[1].min()
        left_lib[key] = left
        top = np.where(markers_lib[key]==2)[0].min()
        top_lib[key] = top
        right = np.where((markers_lib[key]!=1) & (markers_lib[key]!=-1))[1].max()
        right_lib[key] = right

    # Avoid Extracting incorrect ROI
    for key in markers_lib:
        while markers_lib[key][bottom_lib[key]-vert_thres, left_lib[key]+horz_thres] == 1:
            left_lib[key] += horz_thres

    # Corners: Top left, Top right, Bottom left, Bottom right
    src_corners = dict()
    for key in markers_lib:
        corners = []
        tl = [left_lib[key], top_lib[key]]
        tr = [right_lib[key], top_lib[key]]
        bl = [left_lib[key], bottom_lib[key]]
        br = [right_lib[key], bottom_lib[key]]
        corners.append(tl)
        corners.append(tr)
        corners.append(bl)
        corners.append(br)
        src_corners[key] = (corners)

    dst_corners = dict()
    for key in markers_lib:
        dst_corners[key] = get_destination_points(src_corners[key])

#     calibrated = {}
#     for img in img_lib:
#         org = plt.imread(os.path.join(dest, img))
#         org = lab_trans(clear, org)
#         calibrated[img] = org

#     greyscale = {}
#     for img in img_lib:
#         grey = calibrated[img]
#         grey = to_single_channel(grey)
#         greyscale[img] = grey
    
    greyscale = {}
    for img in img_lib:
        grey = plt.imread(os.path.join(dest, img))
        grey = to_single_channel(grey)
        greyscale[img] = grey
    

    deskewed_lib = dict()
    dimensions = dict()
    for key in greyscale:
        deskewed = unwarp(dest, key, np.float32(src_corners[key]), dst_corners[key][0])
        dimensions[key] = (dst_corners[key][1], dst_corners[key][2])
        deskewed_lib[key] = deskewed

    h, w = deskewed_lib['NTU0.jpg'].shape[:2]
    resize_lib = dict()
    resize_dim = dict()
    ref = (dimensions['NTU0.jpg'][0], dimensions['NTU0.jpg'][1])
    for key in deskewed_lib:
        dim = (dimensions[key][0] / ref[0], dimensions[key][1] / ref[1])
        img_dim = (int(h*dim[0]), int(w*dim[1]))
        resize_lib[key] = cv2.resize(deskewed_lib[key], img_dim, interpolation=cv2.INTER_AREA)

    greyscale = {}
    for img in resize_lib:
        grey = resize_lib[img]
        grey = to_single_channel(grey)
        greyscale[img] = grey

    # x_left represents the ROI of the reference, while x_right represents the ROI of the sample
    x_left = []
    x_right = []
    y_left = []
    y_right = []

    # for key in deskewed_lib:
    for key in resize_lib:
        x = []
        x1 = []
        y = []
        y1 = []
        for i in range(dst_corners[key][1], dst_corners[key][1]-vert_thres, -1):
            x.append(i)
            y.append(np.mean(deskewed_lib[key][i-1:i, horz_thres:horz_thres+length]))
        x_left.append(x)
        y_left.append(y)
        for i in range(dst_corners[key][1], dst_corners[key][1]-vert_thres, -1):
            x1.append(i)
            y1.append(np.mean(deskewed_lib[key][i-1:i, horz_thres+dist:
                                             horz_thres+dist+length]))
        x_right.append(x1)
        y_right.append(y1)

    x_left = np.array(x_left)
    x_right = np.array(x_right)
    y_left = np.array(y_left)
    y_right = np.array(y_right)

    # plot the ratio with respect to NTU
    num = []
    for img in img_lib:
        img = img.replace('_', '.')
        a = img.split('NTU')
        a = [b.split('.jpg') for b in a]
        for item in a:
            item.remove("")
        num.append(a)

    NTU = []
    for i in num:
        for j in i:
            if len(j) != 0:
                NTU.append(float(j[0]))

    diff = {}
    for i in range(x_left.shape[0]):
        diff[img_lib[i]] = np.mean(y_right[i] - y_left[i])
        
    diff_c = diff['NTU0.jpg']
    for key in diff:
        diff[key] -= diff_c

    pixel_intensity = []
    for key in diff:
        pixel_intensity.append(diff[key])
    pixel_intensity = np.array(pixel_intensity)

    sorted(diff.items(), key=lambda x:x[0])

    NTU = np.array(NTU)
    theta = np.polyfit(pixel_intensity, NTU, 1)
    
    return theta, h, w, diff_c, ref